In [5]:
import pandas as pd
import sqlite3
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow
import os
from sqlalchemy import create_engine

basedir = os.path.abspath('.')
DATASET = 'kompas'
df = pd.read_csv(f"../static/csv/{DATASET}.csv",index_col = 0) 
df.shape

(2090, 4)

In [6]:
columns = [i.replace(' ', '_') for i in df.columns]
con = sqlite3.connect(f"../db/{DATASET}.db")

#write the pandas dataframe to a sqlite table
df.to_sql(f'{DATASET}',con=con,if_exists='replace',index=True)

#connect to the database
c = con.cursor()

c.executescript(f'''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    
    ALTER TABLE {DATASET} RENAME TO old{DATASET};
    CREATE TABLE {DATASET} (id INTEGER PRIMARY KEY NOT NULL,
                            author TEXT,
                            comment TEXT,
                            line TEXT,
                            true INTEGER);

    INSERT INTO {DATASET} SELECT * FROM old{DATASET};
    DROP TABLE old{DATASET};
    
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
con.close()

In [7]:
con = sqlite3.connect(f"../db/{DATASET}.db")
con.execute(f"PRAGMA table_info({DATASET})").fetchall()



[(0, 'id', 'INTEGER', 1, None, 1),
 (1, 'author', 'TEXT', 0, None, 0),
 (2, 'comment', 'TEXT', 0, None, 0),
 (3, 'line', 'TEXT', 0, None, 0),
 (4, 'true', 'INTEGER', 0, None, 0)]

In [8]:
con.execute(f"SELECT * FROM {DATASET}").fetchall()

[(0,
  'Ardi Ardiansyah',
  'Aquaridendi aminn..munafik modar...',
  'Ardi Ardiansyah: Aquaridendi aminn..munafik modar...',
  0),
 (1,
  'yahya 9',
  'kalo banjir gimana ?? lah mumpung juga gk?',
  'yahya 9: kalo banjir gimana ?? lah mumpung juga gk?',
  0),
 (2,
  'DMC Remix',
  'Itu kucing tua la .gua juga punya kucing besar & mirip kayak gt',
  'DMC Remix: Itu kucing tua la .gua juga punya kucing besar & mirip kayak gt',
  1),
 (3,
  'Yosiaqwqww4 Kuntjorowqeeeh',
  'Bahasa Indonesianya belepotan gitu mau pimpin NKRI..waduh... stand up komedy nih cocoknya....',
  'Yosiaqwqww4 Kuntjorowqeeeh: Bahasa Indonesianya belepotan gitu mau pimpin NKRI..waduh... stand up komedy nih cocoknya....',
  1),
 (4,
  'Embun sang rumangsamu',
  'hhhh.. nampaknya sampean pinter to mbak.. tp kok B pasti dah disodomi itu sama anggota isis.. kasihan akhirnya nyesel dan berdosa dah gak prawan.. kkkkk',
  'Embun sang rumangsamu: hhhh.. nampaknya sampean pinter to mbak.. tp kok B pasti dah disodomi itu sama a